In [79]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Assuming you have your joint angles and EIM data in X and the corresponding labels (joint angle and weight) in y.

# Example data loading (replace this with your actual data loading code)
# X and y should be NumPy arrays
# X should have shape (num_samples, num_timesteps, num_features)
# y should have shape (num_samples, num_output_features)

# Generate example data (replace this with your actual data loading code)
np.random.seed(42)
num_samples = 1000
num_timesteps = 10
num_features = 2  # Replace with the actual number of features for joint angles and EIM data
num_output_features = 2  # Replace with the actual number of output features (joint angle and weight)

X = np.random.rand(num_samples, num_timesteps, num_features)
y = np.random.rand(num_samples, num_output_features)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data using Min-Max scaling
scaler_x = MinMaxScaler(feature_range=(0, 1))
X_train = scaler_x.fit_transform(X_train.reshape(-1, num_features)).reshape(X_train.shape)
X_test = scaler_x.transform(X_test.reshape(-1, num_features)).reshape(X_test.shape)

scaler_y = MinMaxScaler(feature_range=(0, 1))
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# Build the GRU model
model = Sequential()
model.add(GRU(50, activation='relu', input_shape=(num_timesteps, num_features)))
model.add(Dense(num_output_features, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions on new data
# Replace `new_data` with your actual new data
new_data = np.random.rand(1, num_timesteps, num_features)
scaled_new_data = scaler_x.transform(new_data.reshape(-1, num_features)).reshape(new_data.shape)
prediction = model.predict(scaled_new_data)
scaled_prediction = scaler_y.inverse_transform(prediction)
print(f'Predicted values: {scaled_prediction}')

Epoch 1/10
25/25 [==============================] - 1s 16ms/step - loss: 0.1958 - val_loss: 0.1064
Epoch 2/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0908 - val_loss: 0.0853
Epoch 3/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0877 - val_loss: 0.0845
Epoch 4/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0876 - val_loss: 0.0843
Epoch 5/10
25/25 [==============================] - 0s 6ms/step - loss: 0.0868 - val_loss: 0.0836
Epoch 6/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0866 - val_loss: 0.0840
Epoch 7/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0862 - val_loss: 0.0835
Epoch 8/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0864 - val_loss: 0.0850
Epoch 9/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0861 - val_loss: 0.0851
Epoch 10/10
7/7 [==============================] - 0s 3ms/step - loss: 0.0838
Test Loss: 0.08380357921123505
1/1 [===

<h1>Import Libaries</h1>

In [80]:
# %pip install tensorflow
# %pip install pandas
# %pip install scikit-learn
# %pip install plotly
# %pip install scipy

# Tensorflow / Keras
import tensorflow as tf
from tensorflow import keras
print('Tensorflow version:', tf.__version__)
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, GRU, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

import scipy.io
print('scipy: %s' % scipy.__version__) # print version

Tensorflow version: 2.14.0
pandas: 2.1.1
numpy: 1.26.0
sklearn: 1.3.1
plotly: 5.17.0
scipy: 1.11.3


<h1>Load Data</h1>

In [81]:
# Set Pandas options to display more columns
pd.options.display.max_columns=150

# Google drive location weatherset: /content/drive/My Drive/Bachelor/NeuralNetwork/weatherAUS.csv
# Local weatherset location: C:/Users/Simons Lenovo/Desktop/Neural_network_data/weatherAUS.csv
data_dir = "C:/Users/Simons Lenovo/Desktop/Neural_network_data/KIN_MUS_UJI.mat"

# Read in the weather data csv - keep only the columns we need
# For csv read: df=pd.read_csv(data_dir, encoding='utf-8', usecols=['Date', 'Location', 'MinTemp', 'MaxTemp'])
df=scipy.io.loadmat(data_dir)   # Specifically for loading .mat files

# Access variables from the loaded data
# Assuming the struct variable is named 'EMG_KIN_v4'
data_struct = df['EMG_KIN_v4']

# Drops records of NaN
data_struct = data_struct[pd.isnull(data_struct['Kinematic_data'])==False]
data_struct = data_struct[pd.isnull(data_struct['EMG_data'])==False]
data_struct = data_struct[pd.isnull(data_struct['time'])==False]

# Access individual variables from the struct
kinematic_data = data_struct['Kinematic_data']
emg_data = data_struct['EMG_data']
time = data_struct['time']

# Manipulate data as needed for your GRU network
# For example, you might want to reshape time for compatibility with GRU
time_reshaped = time.reshape(-1, 1)  # Assuming 1D time array

# You can now use kinematic_data and emg_data as your features (X) and time as needed
# Ensure that the shapes and dimensions are appropriate for your model

# Print example values
print('Example Kinematic Data:', kinematic_data[0])
print('Example EMG Data:', emg_data[0])
print('Example Time:', time_reshaped[0])

Example Kinematic Data: [[ 10.9143   -45.4987     4.5029   ...  13.4262     0.150621  -0.606393]
 [ 11.0089   -45.5562     4.37224  ...  13.4743     0.320903  -0.624508]
 [ 11.1139   -45.6199     4.23522  ...  13.5422     0.529266  -0.64826 ]
 ...
 [ 13.6956   -28.5612    -7.95915  ...  30.2607    11.4445    -2.18656 ]
 [ 13.7571   -28.4096    -7.81919  ...  30.5908    11.4751    -2.19769 ]
 [ 13.8161   -28.2776    -7.67678  ...  30.9155    11.5116    -2.20595 ]]
Example EMG Data: [[0.0773466 0.0384933 0.0418277 0.0858424 0.0766112 0.170576  0.0309661]
 [0.0791462 0.0389555 0.0426127 0.0895857 0.0785865 0.171149  0.0310779]
 [0.0808208 0.0395307 0.0433776 0.0933961 0.0805707 0.171268  0.0312763]
 [0.082368  0.0402014 0.04413   0.0972621 0.0825621 0.170952  0.0315491]
 [0.0837865 0.0409518 0.0448763 0.101169  0.0845547 0.170223  0.0318832]
 [0.0850765 0.0417672 0.0456211 0.1051    0.0865389 0.169108  0.0322652]
 [0.0862389 0.0426344 0.0463669 0.109033  0.0885006 0.167639  0.0326819]
 [0

<h1>Data preprocessing</h1>

Normalizing. Range of the scaler should be the same for x and y axis

In [111]:
# Assuming you have features X and time
# Normalize the data
scaler = MinMaxScaler(feature_range=(-1,1))

# Normalize each sample (column) in kinematic_data
num_samples = kinematic_data.shape[0]
kinematic_data_normalized = np.zeros_like(kinematic_data)
emg_data_normalized = np.zeros_like(emg_data)

for i in range(num_samples):
    kinematic_data_normalized[i] = scaler.fit_transform(kinematic_data[i])
    emg_data_normalized[i] = scaler.fit_transform(emg_data[i])

emg_data_normalized = emg_data_normalized.flatten()
kinematic_data_normalized = kinematic_data_normalized.flatten()

# Assuming kinematic_data_normalized and emg_data_normalized are NumPy arrays
kinematic_data_normalized = kinematic_data_normalized.astype(float)
emg_data_normalized = emg_data_normalized.astype(float)

print('Example Kinematic Data Normalized:', kinematic_data_normalized[0])
print('Example EMG Data Normalized:', emg_data_normalized[0])
print('Kinematic Data Shape:', kinematic_data_normalized.shape)
print('EMG Data Shape:', emg_data_normalized.shape)
print('Kinematic Data Type:', kinematic_data_normalized.dtype)
print('EMG Data Type:', emg_data_normalized.dtype)

c:\Users\Simons Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
c:\Users\Simons Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:481: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
c:\Users\Simons Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
c:\Users\Simons Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:481: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
c:\Users\Simons Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
c:\Users\Simons Lenovo\AppData\Loca

ValueError: setting an array element with a sequence.

<h1>Prepare sequences for GRU</h1>

In [101]:
def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

# Set the number of time steps (adjust as needed)
time_steps = 10

# Create sequences for kinematic data (assuming it's univariate)
X_kinematic, y_kinematic = create_sequences(kinematic_data_normalized, time_steps)

# Create sequences for EMG data if needed (assuming it's univariate)
X_emg, y_emg = create_sequences(emg_data_normalized, time_steps)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1706,) + inhomogeneous part.

Three months are missing from the dataset, so average from preceding and subsequent months are calculated as placeholder.

In [ ]:

# Add missing months 2011-04, 2011-04, 2011-04 and impute data
df2_pivot['2011-04']=(df2_pivot['2011-03']+df2_pivot['2011-05'])/2
df2_pivot['2012-12']=(df2_pivot['2012-11']+df2_pivot['2013-01'])/2
df2_pivot['2013-02']=(df2_pivot['2013-01']+df2_pivot['2013-03'])/2

# Sort columns so Year-Months are in the correct order
df2_pivot=df2_pivot.reindex(sorted(df2_pivot.columns), axis=1)

Plots the data

In [ ]:
# %pip install nbformat

# Plot average monthly temperature derived from daily medians for each location
fig = go.Figure()
for location in df2_pivot.index:
    fig.add_trace(go.Scatter(x=df2_pivot.loc[location, :].index,
                             y=df2_pivot.loc[location, :].values,
                             mode='lines',
                             name=location,
                             opacity=0.8,
                             line=dict(width=1)
                            ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'), showlegend=True)

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                 showline=True, linewidth=1, linecolor='black',
                 title='Date'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                 showline=True, linewidth=1, linecolor='black',
                 title='Degrees Celsius'
                )

# Set figure title
fig.update_layout(title=dict(text="Average Monthly Temperatures", font=dict(color='black')))

fig.show()

Note how the mean temperature, as well as variation, differs between locations. We can either train a location-specific model for better precision or a generic model to predict temperatures for every area.

In this example, I will create a generic model trained on all locations.

Training and evaluating GRU model Here are a few things to highlight before we start.

We will use sequences of 18 months to predict the average temperatures for the next 18 months. You can adjust that to your liking but beware that there will not be enough data for sequences beyond 23 months in length.

We will split the data into two separate dataframes — one for training and the other for validation (out of time validation).

Since we are creating a many-to-many prediction model, we need to use a slightly more complex encoder-decoder configuration. Both encoder and decoder are hidden GRU layers, with information passed from one to another via a repeat vector layer.

A repeat vector is necessary when we want to have sequences of different lengths, e.g., a sequence of 18 months to predict the next 12 months. It ensures that we provide the right shape for a decoder layer. However, if your input and output sequences are of the same length as in my example, then you can also choose to set return_sequences=True in the encoder layer and remove the repeat vector. Note that we added a Bidirectional wrapper to GRU layers. It allows us to train the model in both directions, which sometimes produces better results. However, its use is optional.

Also, we need to use a Time Distributed wrapper in the output layer to predict outputs for each timestep individually. Finally, I have used MinMaxScaling in this example because it has produced better results than the unscaled version. You can find both scaled and unscaled setups within Jupyter Notebooks in my GitHub repository (link available at the end of the article).

First, let’s define a helper function to reshape the data to a 3D array required by GRU.

In [ ]:
def shaping(datain, timestep, scaler):

    # Loop through each location
    for location in datain.index:
        datatmp = datain[datain.index==location].copy()

        # Convert input dataframe to array and flatten
        arr=datatmp.to_numpy().flatten()

        # Scale using transform (using previously fitted scaler)
        arr_scaled=scaler.transform(arr.reshape(-1, 1)).flatten()

        cnt=0
        for mth in range(0, len(datatmp.columns)-(2*timestep)+1): # Define range
            cnt=cnt+1 # Gives us the number of samples. Later used to reshape the data
            X_start=mth # Start month for inputs of each sample
            X_end=mth+timestep # End month for inputs of each sample
            Y_start=mth+timestep # Start month for targets of each sample. Note, start is inclusive and end is exclusive, that's why X_end and Y_start is the same number
            Y_end=mth+2*timestep # End month for targets of each sample.

            # Assemble input and target arrays containing all samples
            if mth==0:
                X_comb=arr_scaled[X_start:X_end]
                Y_comb=arr_scaled[Y_start:Y_end]
            else:
                X_comb=np.append(X_comb, arr_scaled[X_start:X_end])
                Y_comb=np.append(Y_comb, arr_scaled[Y_start:Y_end])

        # Reshape input and target arrays
        X_loc=np.reshape(X_comb, (cnt, timestep, 1))
        Y_loc=np.reshape(Y_comb, (cnt, timestep, 1))

        # Append an array for each location to the master array
        if location==datain.index[0]:
            X_out=X_loc
            Y_out=Y_loc
        else:
            X_out=np.concatenate((X_out, X_loc), axis=0)
            Y_out=np.concatenate((Y_out, Y_loc), axis=0)

    return X_out, Y_out

Next, we train GRU neural network over 50 epochs and display the model summary with evaluation metrics. You can follow my comments within the code to understand each step.

In [ ]:
##### Step 1 - Specify parameters
timestep=18
scaler = MinMaxScaler(feature_range=(-1, 1))

In [ ]:
##### Step 2 - Prepare data

# Split data into train and test dataframes
df_train=df2_pivot.iloc[:, 0:-2*timestep].copy()
df_test=df2_pivot.iloc[:, -2*timestep:].copy()

# Use fit to train the scaler on the training data only, actual scaling will be done inside reshaping function
scaler.fit(df_train.to_numpy().reshape(-1, 1))

# Use previously defined shaping function to reshape the data for GRU
X_train, Y_train = shaping(datain=df_train, timestep=timestep, scaler=scaler)
X_test, Y_test = shaping(datain=df_test, timestep=timestep, scaler=scaler)

In [ ]:
##### Step 3 - Specify the structure of a Neural Network
model = Sequential(name="GRU-Model") # Model
model.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(Bidirectional(GRU(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False), name='Hidden-GRU-Encoder-Layer')) # Encoder Layer
model.add(RepeatVector(X_train.shape[1], name='Repeat-Vector-Layer')) # Repeat Vector
model.add(Bidirectional(GRU(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False, return_sequences=True), name='Hidden-GRU-Decoder-Layer')) # Decoder Layer
model.add(TimeDistributed(Dense(units=1, activation='linear'), name='Output-Layer')) # Output Layer, Linear(x) = x

In [ ]:
##### Step 4 - Compile the model
model.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance.
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
             )

In [ ]:
##### Step 5 - Fit the model on the dataset
history = model.fit(X_train, # input data
                    Y_train, # target data
                    batch_size=1, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
                    epochs=50, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
                    verbose=1, # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
                    callbacks=None, # default=None, list of callbacks to apply during training. See tf.keras.callbacks
                    validation_split=0.2, # default=0.0, Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch.
                    #validation_data=(X_test, y_test), # default=None, Data on which to evaluate the loss and any model metrics at the end of each epoch.
                    shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
                    class_weight=None, # default=None, Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
                    sample_weight=None, # default=None, Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
                    initial_epoch=0, # Integer, default=0, Epoch at which to start training (useful for resuming a previous training run).
                    steps_per_epoch=None, # Integer or None, default=None, Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined.
                    validation_steps=None, # Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
                    validation_batch_size=None, # Integer or None, default=None, Number of samples per validation batch. If unspecified, will default to batch_size.
                    validation_freq=10, # default=1, Only relevant if validation data is provided. If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs.
                    max_queue_size=10, # default=10, Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
                    workers=1, # default=1, Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
                    use_multiprocessing=True, # default=False, Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False.
                   )

Epoch 1/50
880/880 [==============================] - 22s 13ms/step - loss: 0.0413 - mean_squared_error: 0.0413 - mean_absolute_error: 0.1512
Epoch 2/50
880/880 [==============================] - 14s 16ms/step - loss: 0.0080 - mean_squared_error: 0.0080 - mean_absolute_error: 0.0688
Epoch 3/50
880/880 [==============================] - 15s 17ms/step - loss: 0.0072 - mean_squared_error: 0.0072 - mean_absolute_error: 0.0650
Epoch 4/50
880/880 [==============================] - 16s 18ms/step - loss: 0.0070 - mean_squared_error: 0.0070 - mean_absolute_error: 0.0640
Epoch 5/50
880/880 [==============================] - 11s 13ms/step - loss: 0.0066 - mean_squared_error: 0.0066 - mean_absolute_error: 0.0619
Epoch 6/50
880/880 [==============================] - 13s 15ms/step - loss: 0.0067 - mean_squared_error: 0.0067 - mean_absolute_error: 0.0623
Epoch 7/50
880/880 [==============================] - 13s 15ms/step - loss: 0.0065 - mean_squared_error: 0.0065 - mean_absolute_error: 0.0619
Epoch 

In [ ]:
##### Step 6 - Use model to make predictions
# Predict results on training data
#pred_train = model.predict(X_train)
# Predict results on test data
pred_test = model.predict(X_test)


##### Step 7 - Print Performance Summary
print("")
print('-------------------- Model Summary --------------------')
model.summary() # print model summary
print("")
print('-------------------- Weights and Biases --------------------')
print("Too many parameters to print but you can use the code provided if needed")
print("")
#for layer in model.layers:
#    print(layer.name)
#    for item in layer.get_weights():
#        print("  ", item)
#print("")

# Print the last value in the evaluation metrics contained within history file
print('-------------------- Evaluation on Training Data --------------------')
for item in history.history:
    print("Final", item, ":", history.history[item][-1])
print("")

# Evaluate the model on the test data using "evaluate"
print('-------------------- Evaluation on Test Data --------------------')
results = model.evaluate(X_test, Y_test)
print("")

2/2 [==============================] - 2s 7ms/step

-------------------- Model Summary --------------------
Model: "GRU-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden-GRU-Encoder-Layer (  (None, 64)                6720      
 Bidirectional)                                                  
                                                                 
 Repeat-Vector-Layer (Repea  (None, 18, 64)            0         
 tVector)                                                        
                                                                 
 Hidden-GRU-Decoder-Layer (  (None, 18, 64)            18816     
 Bidirectional)                                                  
                                                                 
 Output-Layer (TimeDistribu  (None, 18, 1)             65        
 ted)                                                            
               

Now, let’s regenerate predictions for the 5 locations we picked earlier and plot the results on a chart to compare actual and predicted values

In [ ]:
# Select locations to predict temperatures for
location=['Cairns', 'Canberra', 'Darwin', 'GoldCoast', 'MountGinini']
dfloc_test = df_test[df_test.index.isin(location)].copy()

# Reshape test data
X_test, Y_test = shaping(datain=dfloc_test, timestep=timestep, scaler=scaler)

# Predict results on test data
pred_test = model.predict(X_test)

1/1 [==============================] - 0s 34ms/step


Plot results

In [ ]:
# Plot average monthly temperatures (actual and predicted) for test (out of time) data
fig = go.Figure()

# Trace for actual temperatures
for location in dfloc_test.index:
    fig.add_trace(go.Scatter(x=dfloc_test.loc[location, :].index,
                             y=dfloc_test.loc[location, :].values,
                             mode='lines',
                             name=location,
                             opacity=0.8,
                             line=dict(width=1)
                            ))

# Trace for predicted temperatures
for i in range(0,pred_test.shape[0]):
    fig.add_trace(go.Scatter(x=np.array(dfloc_test.columns[-timestep:]),
                             # Need to inverse transform the predictions before plotting
                             y=scaler.inverse_transform(pred_test[i].reshape(-1,1)).flatten(),
                             mode='lines',
                             name=dfloc_test.index[i]+' Prediction',
                             opacity=1,
                             line=dict(width=2, dash='dot')
                            ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                 showline=True, linewidth=1, linecolor='black',
                 title='Year-Month'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                 showline=True, linewidth=1, linecolor='black',
                 title='Degrees Celsius'
                )

# Set figure title
fig.update_layout(title=dict(text="Average Monthly Temperatures", font=dict(color='black')))
fig.show()